In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%cd "../.."

/home/hazzu/Code/opencv_streamlit


In [3]:
import numpy as np

pr_sift = np.load("./services/semantic_keypoint_detection/pr_sift.npy")
pr_orb = np.load("./services/semantic_keypoint_detection/pr_orb.npy")

pr_sift_fulls = []
pr_orb_fulls = []
for i in range(8):
    pr_sift_fulls.append(
        np.where(np.logical_and(pr_sift[i][:, 0] == 1, pr_sift[i][:, 1] == 1))[0]
    )
    pr_orb_fulls.append(
        np.where(np.logical_and(pr_orb[i][:, 0] == 1, pr_orb[i][:, 1] == 1))[0]
    )

In [7]:
import os
import cv2
from services.semantic_keypoint_detection.services import DATATYPES


accuracy_rotations_orb = []
orb = cv2.ORB_create()
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

for alpha in range(0, 360, 10):
    accuracy_orb = []
    for i in range(8):
        for id in pr_orb_fulls[i]:
            # Read image
            image = cv2.imread(os.path.join(DATATYPES[i], "images", f"{id}.png"))

            # Rotate image
            rotate_matrix = cv2.getRotationMatrix2D(
                ((image.shape[1] - 1) / 2.0, (image.shape[0] - 1) / 2.0), alpha, 1.0
            )
            image_rotated = cv2.warpAffine(
                image.copy(), rotate_matrix, (image.shape[1], image.shape[0])
            )

            # Convert to gray
            gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            gray_image_rotated = cv2.cvtColor(image_rotated, cv2.COLOR_BGR2GRAY)

            # Detect keypoints
            original_keypoints, original_descriptions = orb.detectAndCompute(
                gray_image, None
            )

            rotated_keypoints, rotated_descriptions = orb.detectAndCompute(
                gray_image_rotated, None
            )

            if original_descriptions is None or rotated_descriptions is None:
                accuracy_orb.append(0)
            else:
                matches = bf.match(original_descriptions, rotated_descriptions)
                accuracy_orb.append(
                    len(matches)
                    / max(len(original_descriptions), len(rotated_descriptions))
                )

    accuracy_rotations_orb.append(np.mean(accuracy_orb))

In [8]:
print(accuracy_rotations_orb)

[np.float64(1.0), np.float64(0.7118986956452131), np.float64(0.6489939897771113), np.float64(0.5823726228814303), np.float64(0.5332206234121253), np.float64(0.5064483085921113), np.float64(0.46996367280269025), np.float64(0.4148382549979112), np.float64(0.4039704385149031), np.float64(0.42971605444008565), np.float64(0.4262422969221164), np.float64(0.4499190920700475), np.float64(0.48575494467455615), np.float64(0.5331637974020808), np.float64(0.5860344958867193), np.float64(0.6262579590221845), np.float64(0.6804072697603177), np.float64(0.7345108113183805), np.float64(1.0), np.float64(0.7107193768796876), np.float64(0.6459539838060709), np.float64(0.582874087835228), np.float64(0.532891067910156), np.float64(0.5045699056883001), np.float64(0.4691973892777861), np.float64(0.41414737049363926), np.float64(0.4045218411352713), np.float64(0.42971605444008565), np.float64(0.4265022668372587), np.float64(0.44934437942636946), np.float64(0.48552854272401624), np.float64(0.5346127698855359), 

In [5]:
import os
import cv2
from services.semantic_keypoint_detection.services import DATATYPES


accuracy_rotations_sift = []
sift = cv2.SIFT_create()
bf = cv2.BFMatcher()

for alpha in range(0, 360, 10):
    accuracy_sift = []
    for i in range(8):
        for id in pr_sift_fulls[i]:
            # Read image
            image = cv2.imread(os.path.join(DATATYPES[i], "images", f"{id}.png"))

            # Rotate image
            rotate_matrix = cv2.getRotationMatrix2D(
                ((image.shape[1] - 1) / 2.0, (image.shape[0] - 1) / 2.0), alpha, 1.0
            )
            image_rotated = cv2.warpAffine(
                image.copy(), rotate_matrix, (image.shape[1], image.shape[0])
            )

            # Convert to gray
            gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            gray_image_rotated = cv2.cvtColor(image_rotated, cv2.COLOR_BGR2GRAY)

            # Detect keypoints
            original_keypoints, original_descriptions = sift.detectAndCompute(
                gray_image, None
            )

            rotated_keypoints, rotated_descriptions = sift.detectAndCompute(
                gray_image_rotated, None
            )

            if original_descriptions is None or rotated_descriptions is None:
                accuracy_sift.append(0)
            else:
                matches = bf.match(original_descriptions, rotated_descriptions)

                # good_matches = 0
                # for k in range(len(matches)):
                #     if len(matches[k]) == 2:
                #         if matches[k][0].distance < 0.75 * matches[k][1].distance:
                #             good_matches += 1

                accuracy_sift.append(
                    len(matches)
                    / max(len(original_descriptions), len(rotated_descriptions))
                )

    accuracy_rotations_sift.append(np.mean(accuracy_sift))

In [6]:
print(accuracy_rotations_sift)

[np.float64(1.0), np.float64(0.7178840215294234), np.float64(0.5934762435588593), np.float64(0.6968441162659103), np.float64(0.7022090282353878), np.float64(0.700128314806944), np.float64(0.6772326828561296), np.float64(0.5306231042522277), np.float64(0.642743634619425), np.float64(0.7700695465815599), np.float64(0.6771707418973368), np.float64(0.5615641748276554), np.float64(0.6740924196299639), np.float64(0.6235028273760669), np.float64(0.670448565518988), np.float64(0.6589999043553963), np.float64(0.5475350603178032), np.float64(0.696981033968672), np.float64(0.956118529357966), np.float64(0.7060334239083206), np.float64(0.5791748725763501), np.float64(0.7086376394382324), np.float64(0.6922701091284574), np.float64(0.7089708942442996), np.float64(0.6624140896186212), np.float64(0.5259435114403322), np.float64(0.6344661002234149), np.float64(0.753146370337754), np.float64(0.6880778765099395), np.float64(0.568413023617624), np.float64(0.6743283769675869), np.float64(0.6383578048089886

In [7]:
np.save(
    os.path.join("./services/keypoint_matching/accuracy_rotations_sift.npy"),
    accuracy_rotations_sift,
)
# np.save(
#     os.path.join("./services/keypoint_matching/accuracy_rotations_orb.npy"),
#     accuracy_rotations_orb,
# )